In [64]:
pip install --upgrade IPython 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [65]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [66]:
pip install pandas scikit-learn beautifulsoup4 matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [67]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [69]:

# تحميل البيانات
train_data = pd.read_csv('data/kg_train.csv')
test_data = pd.read_csv('data/kg_test.csv')
# Print dataset shapes
print("Training set shape:", train_data.shape)
print("Testing set shape:", test_data.shape)

# عرض أول 5 صفوف للتأكد من البيانات
print(train_data.head())

Training set shape: (5964, 2)
Testing set shape: (5964, 1)
                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0


### Let's divide the training and test set into two partitions

In [70]:
# Your code

# تقسيم بيانات التدريب إلى 80% تدريب و 20% اختبار
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_train_tfidf, train_data['label'], test_size=0.2, random_state=42)


## Data Preprocessing

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [71]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [72]:

# Your code
# تنظيف النصوص (إزالة HTML و CSS)
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # إزالة HTML
    text = re.sub(r'http\S+', '', text)  # إزالة الروابط
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # إزالة الأرقام والعلامات
    return text.lower()  # تحويل النص إلى حروف صغيرة

train_data['cleaned_text'] = train_data['text'].apply(clean_text)
test_data['cleaned_text'] = test_data['text'].apply(clean_text)


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [73]:
# Your code

## Now let's work on removing stopwords
Remove the stopwords.

In [74]:
# Your code
# إزالة stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

train_data['cleaned_text'] = train_data['cleaned_text'].apply(remove_stopwords)
test_data['cleaned_text'] = test_data['cleaned_text'].apply(remove_stopwords)



[nltk_data] Downloading package stopwords to C:\Users\وجدان
[nltk_data]     ناصر/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [75]:
# Your code
# Stemming: تحويل الكلمات إلى جذرها
stemmer = PorterStemmer()

def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

train_data['cleaned_text'] = train_data['cleaned_text'].apply(stem_text)
test_data['cleaned_text'] = test_data['cleaned_text'].apply(stem_text)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [76]:
# Your code
# اكتشاف أكثر 10 كلمات شيوعًا في ham و spam
ham_words = train_data[train_data['label'] == 'ham']['cleaned_text'].str.split().explode().value_counts().head(10)
spam_words = train_data[train_data['label'] == 'spam']['cleaned_text'].str.split().explode().value_counts().head(10)

print("Top 10 ham words:\n", ham_words)
print("Top 10 spam words:\n", spam_words)

# تحويل النصوص إلى تمثيل عددي (Bag of Words)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['cleaned_text'])
X_test = vectorizer.transform(test_data['cleaned_text'])


Top 10 ham words:
 Series([], Name: count, dtype: int64)
Top 10 spam words:
 Series([], Name: count, dtype: int64)


## Extra features

In [78]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
'''money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()'''

'money_simbol_list = "|".join(["euro","dollar","pound","€","$"])\nsuspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])\n\ndata_train[\'money_mark\'] = data_train[\'preprocessed_text\'].str.contains(money_simbol_list)*1\ndata_train[\'suspicious_words\'] = data_train[\'preprocessed_text\'].str.contains(suspicious_words)*1\ndata_train[\'text_len\'] = data_train[\'preprocessed_text\'].apply(lambda x: len(x)) \n\ndata_val[\'money_mark\'] = data_val[\'preprocessed_text\'].str.contains(money_simbol_list)*1\ndata_val[\'suspicious_words\'] = data_val[\'preprocessed_text\'].str.contains(suspicious_words)*1\ndata_val[\'text_len\'] = data_val[\'preprocessed_text\'].apply(lambda x: len(x)) \n\ndata_train.head()'

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

# تحويل إلى TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['cleaned_text'])

## And the Train a Classifier?

In [ ]:
# Your code

# قائمة النماذج التي سيتم تجربتها
models = [
    ('Naive Bayes', MultinomialNB()),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('SVM', SVC(random_state=42)),
    ('Logistic Regression', LogisticRegression(random_state=42))
]
best_model= None
best_accuracy=0

# استخدام for loop لتدريب واختبار النماذج
for model_name, model in models:
    print(f"Training and evaluating {model_name}...")
    
    # تدريب النموذج
    model.fit(X_train_split, y_train_split)
    
    # التنبؤ على بيانات اختبار التدريب (20% من بيانات التدريب)
    predictions_train = model.predict(X_test_split)
    
    # تقييم النموذج على بيانات اختبار التدريب
    train_accuracy = accuracy_score(y_test_split, predictions_train)
    print(f"{model_name} - Training Accuracy: {train_accuracy * 100:.2f}%")
    if train_accuracy >best_accuracy:
           best_accuracy = train_accuracy
           best_model=model
    print("-" *50)
    if best_model is not None:
       print(f"\nBest model: {best_model.__class__.__name__} with accuracy: {best_accuracy * 100:.2f}%")
    else:
        print("\nNo model was selected. Check the training process.")
    # طباعة أفضل نموذج تم اختياره
print(f"\nBest model: {best_model._class.name_} with accuracy: {best_accuracy * 100:.2f}%")


# استخدام أفضل نموذج للتنبؤ على بيانات kg_test.csv
predictions_test = best_model.predict(X_test_tfidf)

# طباعة أول 10 تنبؤات فقط
print("\nPredictions for kg_test.csv (first 10 rows):")
print(predictions_test[:10])

# حفظ التوقعات في ملف CSV جديد
test_data['predicted_label'] = predictions_test
test_data[['text', 'predicted_label']].to_csv("kg_test_predictions.csv", index=False)

print("\nPredictions saved to kg_test_predictions.csv")
    

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code